In [1]:
# .env 파일을 읽어서 환경변수로 설정
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

True

In [2]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

os.environ['LANGCHAIN_PROJECT'] = 'Runnable'
print(f"[LANGCHAIN_PROJECT]\n{os.environ['LANGCHAIN_PROJECT']}")

[LANGCHAIN_PROJECT]
Runnable


In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Runnable")

LangSmith 추적을 시작합니다.
[프로젝트명]
Runnable


## Langchain에서 데이터를 효과적으로 전달하는 방법
1. RunnablePassthrough
2. RunnableParallel
3. RunnableLambda


### 1. `RunnablePassthrough`: 데이터를 그대로 넘겨(통과시켜)주는 역할

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template("{word}를 영어로?")
llm = ChatOpenAI(temperature=0)

# chain 을 생성합니다.
chain = prompt | llm
chain

PromptTemplate(input_variables=['word'], input_types={}, partial_variables={}, template='{word}를 영어로?')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f263ecbd060>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f263ecbf160>, root_client=<openai.OpenAI object at 0x7f2664188f10>, root_async_client=<openai.AsyncOpenAI object at 0x7f263ecbd0c0>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

chain 을 `invoke()` 하여 실행할 때는 입력 데이터의 타입은 ***딕셔너리***

In [5]:
# chain 을 실행합니다.
chain.invoke({"word": '사과'})

AIMessage(content='apple', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ab2c658b-b372-4038-bb63-d3f8ef5cde4d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

But, 1개의 변수만 템플릿에 포함하고 있다면, 값만 전달하는 것도 가능

In [6]:
# chain 을 실행합니다.
chain.invoke('사과')

AIMessage(content='apple', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-89e8faca-fb40-4f40-9e96-66f0a0fa51e7-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [7]:
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough 는 runnable 객체이며, runnable 객체는 invoke() 메소드를 사용하여 별도 실행이 가능
# RunnablePassthrough()를 사용하여 체인 구성
runnable_chain = {"word": RunnablePassthrough()} | prompt | ChatOpenAI()

# dict 값이 RunnablePassthrough() 로 변경됨
runnable_chain.invoke('인공지능')

AIMessage(content='Artificial Intelligence (AI)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 17, 'total_tokens': 23, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1ac3cc99-5891-4f9e-9c79-4f89cb008bf1-0', usage_metadata={'input_tokens': 17, 'output_tokens': 6, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### 2. `RunnableParallel`: 여러 작업을 동시에(병렬)로 처리하도록 도와주는 도구

In [8]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [9]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='대한민국의 수도는 서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 19, 'total_tokens': 34, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b87f0402-9e88-4ffe-8d20-f1f863565eb5-0', usage_metadata={'input_tokens': 19, 'output_tokens': 15, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}),
 'area': AIMessage(content='대한민국의 면적은 약 100,363km² 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 20, 'total_tokens': 42, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'mod

### 3. `RunnableLambda`: 사용자 정의 함수 매핑하도록 도와주는 도구


In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def concat_output(text):
    return text['capital'].content + ' ' + text['area'].content

In [13]:
final_chain = (combined_chain
            | {'info': RunnableLambda(concat_output)} 
            | PromptTemplate.from_template("{info}의 내용을 자연스럽게 다듬고, 이모지를 넣어줘.")
            | ChatOpenAI(model='gpt-4o-mini'))

final_chain.invoke("대한민국")

AIMessage(content='대한민국의 수도는 서울입니다. 🇰🇷 대한민국의 면적은 약 100,363.4 km²에 달합니다. 🌏✨', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 48, 'total_tokens': 81, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-afd75a3a-f0d7-4abe-b6f8-5c72a77b9b82-0', usage_metadata={'input_tokens': 48, 'output_tokens': 33, 'total_tokens': 81, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime


def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)

'Oct-14'

In [15]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 대한민국 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
# 출력
print(chain.invoke(3))

다음은 10월 14일에 생일인 대한민국의 유명인 3명입니다:

1. **이정재** - 1973년 10월 14일
2. **김장훈** - 1971년 10월 14일
3. **이상민** - 1973년 10월 14일

이 외에도 다른 유명인들이 있을 수 있습니다.


### [실습] Runnnable 세가지를 적절히 사용하여 챗봇 생성

- 위 코드를 활용하여 아래 내용에 맞게 작성하시요.
- RunnableParallel 사용
    - chain1 : `{food}` 재료로 만들 수 있는 메인 디시를 묻는 체인
    - chain2 : `{food}` 재료로 만들 수 있는 후식 디저트를 묻는 체인
- RunnableLambda 사용
    - 두 체인의 결과 값 이어 붙이기 -> `{info}`
    - `{info}`를 바탕으로 이모지를 사용하여 우리 음식점 메뉴 추천 홍보문구를 작성해주는 템플릿


In [26]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 첫 번째 체인: 재료로 만들 수 있는 메인 디시를 묻는 체인
chain1 = (
    {"fordish": RunnablePassthrough(), 'n':RunnablePassthrough()}
    | PromptTemplate.from_template("{fordish}로(으로) 만들 수 있는 메인 요리 {n}가지는?")
    | ChatOpenAI()
)

# 두 번째 체인: 재료로 만들 수 있는 후식 디저트를 묻는 체인
chain2 = (
    {"fordish": RunnablePassthrough(), 'n':RunnablePassthrough()}
    | PromptTemplate.from_template("{fordish}로(으로) 만들 수 있는 디저트 {n}가지는?")
    | ChatOpenAI()
)

# print(chain1.invoke({'fordish':'양파', 'n':3}))
# print(chain2.invoke({'fordish':'양파', 'n':3}))

# 두 체인을 병렬로 실행
combined_chain = RunnableParallel(maindish=chain1, dessert=chain2)

# 두 체인의 결과를 이어 붙이는 함수
def concat_food_info(text):
    return text['maindish'].content + ' ' + text['dessert'].content

# {info}를 바탕으로 이모지를 사용하여 우리 음식점 메뉴 추천 홍보문구를 작성해주는 템플릿
prompt = """
{info}를 바탕으로 이모지를 사용하여 우리 음식점을 소개하는 것처럼 각 메뉴마다 추천 홍보문구를 작성해줘.

#FORMAT:
- 메인 요리 :
- 디저트 :
"""

# 최종 체인은 두 체인의 결과를 이어 붙이고 우리 가게 홍보 문구를 생성하는 작업
final_chain = (combined_chain
            | {'info': RunnableLambda(concat_food_info)} 
            | PromptTemplate.from_template(prompt)
            | ChatOpenAI(model='gpt-4o-mini')
            | StrOutputParser())
            

# 사용자가 생성할 메뉴의 개수를 입력받아 처리
final_output = final_chain.invoke({'fordish':'양파', 'n':3})

# 결과 출력
print(final_output)


- 메인 요리 : 🥘 양파볶음밥 - 고소한 양파의 풍미가 가득 담긴 볶음밥! 🍚 한 입 먹으면 입안 가득 행복이 퍼져요! 😊  
- 디저트 : 🍰 양파 케이크 - 양파의 새로운 변신! 달콤하고 부드러운 케이크가 당신의 미각을 사로잡습니다! 🍮✨  

- 메인 요리 : 🍟 양파튀김 - 바삭바삭한 양파튀김, 겉은 바삭하고 속은 촉촉! 간편하게 즐기는 최고의 간식! 😋  
- 디저트 : 🥧 양파 타르트 - 달콤한 양파의 맛이 조화를 이루는 타르트! 한 조각으로 행복한 시간! 🌟  

- 메인 요리 : 🍲 양파스프 - 깊고 진한 맛의 양파스프, 따뜻하게 몸을 녹여주는 최고의 선택! ☕❤️  
- 디저트 : 🍦 양파 아이스크림 - 신선한 양파로 만든 특별한 아이스크림! 색다른 맛의 여행을 떠나보세요! 🌈🍧


In [ ]:
final_output